<a href="https://colab.research.google.com/github/Saikrishnaraju0510/bigquery/blob/main/BATCH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apache-beam[gcp]

     |████████████████████████████████| 9.9 MB 4.4 MB/s 
     |████████████████████████████████| 151 kB 47.5 MB/s 
     |████████████████████████████████| 245 kB 61.5 MB/s 
     |████████████████████████████████| 2.3 MB 64.3 MB/s 
     |████████████████████████████████| 508 kB 62.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 435 kB 60.0 MB/s 
     |████████████████████████████████| 183 kB 65.6 MB/s 
     |████████████████████████████████| 83 kB 2.1 MB/s 
     |████████████████████████████████| 172 kB 71.6 MB/s 
     |████████████████████████████████| 255 kB 65.6 MB/s 
     |████████████████████████████████| 144 kB 65.8 MB/s 
     |████████████████████████████████| 173 kB 63.0 MB/s 
     |████████████████████████████████| 180 kB 59.3 MB/s 
     |████████████████████████████████| 267 kB 65.0 MB/s 
     |████████████████████████████████| 111 kB 62.8 MB/s 
     |███████████████

In [1]:
!pip install google-apitools

In [7]:
from apache_beam.io.gcp.bigquery import WriteToBigQuery
import argparse
import os
from datetime import date
from google.cloud import bigquery
path1='key.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path1
tab_id='airflow4.cashpayments'
tab_id2='airflow4.walletpayments'
tab_id1='airflow4.cardpayments'
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

parser = argparse.ArgumentParser()
# parser.add_argument('--my-arg')
args, beam_args = parser.parse_known_args()

# Create and set your Pipeline Options.
beam_options = PipelineOptions(
    beam_args,
    runner='DataflowRunner',
    temp_location='gs://us-central1-c1-dea909ea-bucket/temp1',
    service_account_email='demo-py@modular-asset-332406.iam.gserviceaccount.com',
    region='us-central1',
    project='modular-asset-332406',
    stage_location='gs://sai_1999/stage'
    )
#args = beam_options.view_as(MyOptions)
#today= 2548566
client = bigquery.Client()

dataset_id = "modular-asset-332406.airflow4"

try:
	client.get_dataset(dataset_id)

except:
	dataset = bigquery.Dataset(dataset_id)  #

	dataset.location = "US"
	dataset.description = "dataset"

	dataset_ref = client.create_dataset(dataset)
def to_json(fields):
    json_str = {"Invoice_ID":fields[0],
                 "Branch": fields[1],
                 "City": fields[2],
                 "Customer_type": fields[3],
                 "Gender": fields[4],
                 "Product_line": fields[5],
                 "Unit_price": fields[6],
                 "Quantity": fields[7],
                 "Tax_5": fields[8],
                 "Total": fields[9],
                 "Date": '256485',
                 "Time": fields[11],
                 "Payment": fields[12],
                 "gross_income": fields[15],
                 "Rating":fields[16]
                 }
    return json_str
schema_1='Invoice_ID:STRING,Branch:STRING,City:STRING,Customer_type:STRING,Gender:STRING,Product_line:STRING,Unit_price:FLOAT,Quantity:INTEGER,Tax_5:FLOAT,Total:FLOAT,Date:STRING,Time:STRING,Payment:STRING,gross_income:STRING,Rating:STRING'


p=beam.Pipeline(options=beam_options)
data_ingestion=(
    p
    | beam.io.ReadFromText('gs://sai_1999/supermarket_sales.csv',skip_header_lines=1)
    | beam.Map(lambda x:x.split(','))
)
cash_payments=(
    data_ingestion
    | beam.Filter(lambda x:x[12]=="Cash")
    | "FORMAT">>beam.Map(to_json)
    | "write to cash table">> beam.io.WriteToBigQuery(
        tab_id,
        schema=schema_1,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
	      write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
)
wallet_payments=(
    data_ingestion
    | "filter">>beam.Filter(lambda x:x[12]=="Ewallet")
    | "TO_JSON">>beam.Map(to_json)
    | "write to wallet table">> beam.io.WriteToBigQuery(
        tab_id2,
        schema=schema_1,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
	      write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
)
card_payments=(
    data_ingestion
    | "filter2">>beam.Filter(lambda x:x[12]=="Credit card")
    | "WTBQ">>beam.Map(to_json)
    | "write to card table">> beam.io.WriteToBigQuery(
        tab_id1,
        schema=schema_1,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
	      write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
)
p.run()

/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery.py:2102: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location


<DataflowPipelineResult <Job
 clientRequestId: '20220127071149911073-3925'
 createTime: '2022-01-27T07:11:55.460224Z'
 currentStateTime: '1970-01-01T00:00:00Z'
 id: '2022-01-26_23_11_54-6960039337511878359'
 location: 'us-central1'
 name: 'beamapp-root-0127071149-909757'
 projectId: 'modular-asset-332406'
 stageStates: []
 startTime: '2022-01-27T07:11:55.460224Z'
 steps: []
 tempFiles: []
 type: TypeValueValuesEnum(JOB_TYPE_BATCH, 1)> at 0x7f32b6b9a890>